In [ ]:
import os
import pandas as pd

In [ ]:
# https://ec.europa.eu/eurostat/databrowser/view/env_air_gge__custom_15823458/default/table?lang=en
# Unit - kt
national_cars_de_2019 = 101037.24
national_motorcycles_de_2019 = 1436.32
national_ldv_de_2019 = 12169.85
national_hdv_and_buses_de_2019 = 44900.68

national_cars_es_2019 = 55500.2
national_motorcycles_es_2019 = 1876.82
national_ldv_es_2019 = 6852.57
national_hdv_and_buses_es_2019 = 20305.87

In [ ]:
national_road_transport_de_2019 = national_cars_de_2019 + national_motorcycles_de_2019 + national_ldv_de_2019 + national_hdv_and_buses_de_2019
national_road_transport_es_2019 = national_cars_es_2019 + national_motorcycles_es_2019 + national_ldv_es_2019 + national_hdv_and_buses_es_2019

In [ ]:
disagg_road_cars_2022 = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_transport_using_cars.csv")

disagg_road_motorcycles_2022 = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_transport_using_motorcycles.csv")

disagg_road_freight_2022 = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_freight_transport.csv")

In [ ]:
disagg_road_2022 = pd.merge(disagg_road_cars_2022, disagg_road_motorcycles_2022, on="region_code")
disagg_road_2022["value"] = disagg_road_2022["value_x"] + disagg_road_2022["value_y"]

disagg_road_2022.drop(columns=["value_confidence_level_x", "value_x", "value_confidence_level_y", "value_y"], inplace=True)

disagg_road_2022 = pd.merge(disagg_road_2022, disagg_road_freight_2022, on="region_code")

disagg_road_2022["value"] = disagg_road_2022["value_x"] + disagg_road_2022["value_y"]

disagg_road_2022.drop(columns=["value_confidence_level", "value_x", "value_y"], inplace=True)

In [ ]:
disagg_road_de_2019 = disagg_road_2022[disagg_road_2022["region_code"].str.startswith("DE")].copy()
disagg_road_es_2019 = disagg_road_2022[disagg_road_2022["region_code"].str.startswith("ES")].copy()

In [ ]:
disagg_road_de_2019["share"] = disagg_road_de_2019["value"] / disagg_road_de_2019["value"].sum()
disagg_road_es_2019["share"] = disagg_road_es_2019["value"] / disagg_road_es_2019["value"].sum()

In [ ]:
disagg_road_de_2019["disagg_value"] = disagg_road_de_2019["share"] * national_road_transport_de_2019
disagg_road_es_2019["disagg_value"] = disagg_road_es_2019["share"] * national_road_transport_es_2019

In [ ]:
code_name_mapping = {"ES300_28079": "Madrid",
                    "ES418_47186": "Valladolid",
                   "ES523_46250":  "València",
                    "ES211_01059": "Vitoria-Gasteiz",
                   "ES243_50297":  "Zaragoza",
                    "ES618_41091": "Sevilla",
                    "ES511_08019": "Barcelona", 
                    "DE125_08221000": "Heidelberg"}

In [ ]:
disagg_values = disagg_road_es_2019[disagg_road_es_2019["region_code"].isin(code_name_mapping.keys())]

In [ ]:
disagg_values["region_name"] = disagg_values["region_code"].map(code_name_mapping)

In [ ]:
CCC_values = {"Madrid": 2315.24,
                    "Valladolid": 231.99,
                   "València": 513.58,
                    "Vitoria-Gasteiz": 197.26,
                   "Zaragoza": 480.67,
                    "Sevilla": 789.59,
                    "Barcelona": 906}

In [ ]:
disagg_values["CCC_value"] = disagg_values["region_name"].map(CCC_values)

In [ ]:
disagg_values_clean = disagg_values[["region_name", "CCC_value", "disagg_value"]]

In [ ]:
disagg_values_clean.to_csv(os.path.join("..", "..", "..", "..", "data", "disaggregated_data", "validation", "road_transport_emissions.csv"), index=False)